In [2]:
import os
import requests
import os
import sys
from tqdm import tqdm
import time
import Cobol2Spark as chat
import numpy as np

In [3]:
with open('DWSD0612_completo_V3.txt', 'r') as file:
    data = file.read().strip()
    data = ' '.join(data.split())

In [4]:
final = []
divisions = ["ENVIRONMENT DIVISION", "DATA DIVISION","PROCEDURE DIVISION"]
data_split = data.split('DIVISION')[2:]
for i,row in enumerate(data_split):
    final.append(f'{divisions[i]} {row}')


In [5]:
usage = []
explains = []
responses = []
for code in tqdm(final):
    explanation = chat.generate(prompt=f"""Given the following code snippet. Your role will be to decipher the code, create an in-depth explanation of code providing all functionality, including all the necessary processes.

    code:
    ```
    {code}
    ```

    Your task is to do the following steps:

    1. Analyze and comprehend the COBOL code snippet above.
    2. Write an in-depth step by step Explanation of code.
    3. Documentaion should provide detailed step by step with all proccess made in code.
    4. pay attention in all data input and output steps like data reading, data writing, and data loading.
    4. Pay extra attention in the every data processing steps like filtering, aggregation, and sorting.
    5. Pay attention in every data manipulation steps like data cleaning, data transformation, and data wrangling.
    """,temperature=0.4, max_tokens=1000)

    code = chat.generate(prompt=f"""Given the following code snippet and the explaination of code Your role will be to decipher the code, create an  PySpark code with the same functionality, including all the necessary processes.

    explanation:
    ```
    {explanation.choices[0].message.content}
    ```
    code:
    ```
    {code}
    ```

    Your task is to do the following steps:

    1. Analyze and comprehend the COBOL code snippet above.
    2. Write an equivalent PySpark code.
    3. The equivalent code should provide the exact same output as the COBOL code.
    4. Pay extra attention in the every data processing steps like filtering, aggregation, and sorting.
    5. Follow best practices while crafting your code, like writing clean, well-indented, efficient, and concise code.
    6. Return only the PySpark code.
    """,temperature=0.4, max_tokens=1000)
    responses.append(response)
    usage.append(((response.usage.prompt_tokens/1000) * 0.006)+((response.usage.completion_tokens/1000) * 0.012))
print(np.sum(usage))

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [02:15<00:00, 45.12s/it]

0.158646


In [6]:
documentation = '\n\n'.join(resp.choices[0].message.content for resp in responses)

In [11]:
print(responses[1].choices[0].message.content)

The given COBOL code is quite extensive and involves many steps. Let's break it down into sections and explain each part:

1. **File and Data Definitions**: The code starts with the definition of several files (ARQPARM, ARQSORT, DIMESTIP, ARQFTP) and their associated data structures. Each file has a specific layout defined using the PIC clause, which describes the type and size of each field. For instance, ARQSORT contains fields like NUM-CGC-SORT, DATA-CANCEL-SORT, DATA-INIC-SORT, and others. 

2. **Working Storage Section**: This section defines various temporary variables and constants that will be used throughout the program. For example, WS-SDCDSUC, WS-ABEND, WS-SQLCODE, etc. These variables are used for different purposes such as error handling, storing intermediate results, and controlling program flow.

3. **SQL Statements and Table Definitions**: The code includes several EXEC SQL statements that interact with a DB2 database. These statements include table definitions (e.g., A

In [7]:
import re

sparks = []
for resp in  responses:
    code = '\n\n'.join(re.findall(r'```python\n(.*?)```', resp.choices[0].message.content, re.DOTALL))
    sparks.append(code) 
final_sparks = '\n\n\n '.join(sparks)

In [ ]:
cobol_final = '\n\n'.join(final)

In [ ]:
final_code = chat.generate(prompt=f"""Analise the following codes snippet and decipher the code.
                            

cobol code:
```
{cobol_final}
```

Pyspark code:
```
{final_sparks}
```

Your task is to:

1. Analyze and comprehend the PySpark and cobol code snippet above.
2. verify if in the Spark code there is any mistake or if it is not equivalent to the COBOL code.
3. If there is any mistake, correct it.
4. If the code is missing some functionality, add it.
5. pay extra attention to the data types and the every data processing steps like filtering, aggregation, and sorting.
6. Return only the PySpark code.""",temperature=0.4,max_tokens=1000)

print(((final_code.usage.prompt_tokens/1000) * 0.006)+((final_code.usage.completion_tokens/1000) * 0.012))

In [ ]:
print(final_code.choices[0].message.content)